# Capstone Project - Finnish...

## Assignment Instructions from the course

Coursera Course: _Applied Data Science Capstone_

You have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to *explore or compare neighborhoods or cities*
of your choice **or** to *come up with a problem that you can use the Foursquare location data to solve*. If you cannot think of an idea or a problem, here are some ideas to get you started:

1. In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.

2. In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?

These are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.


For the first week, you will required to submit the following:

1. A description of the problem and a discussion of the background. (15 marks)
2. A description of the data and how it will be used to solve the problem. (15 marks)


For the second week, the final deliverables of the project will be:

1. A link to your Notebook on your Github repository, showing your code. (15 marks)
2. A full report consisting of all of the following components (15 marks):
    - **Introduction** where you discuss the business problem and who would be interested in this project.
    - **Data** where you describe the data that will be used to solve the problem and the source of the data.
    - **Methodology** section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.
    - **Results** section where you discuss the results.
    - **Discussion** section where you discuss any observations you noted and any recommendations you can make based on the results.
    - **Conclusion** section where you conclude the report.
3. Your choice of a presentation or blogpost. (10 marks)


## Working notes

Wikipedia:
- Luettelo Suomen kunnista, sisältäen mm. väkiluku ja pinta-ala: https://fi.wikipedia.org/wiki/Luettelo_Suomen_kunnista
- Luettelo Suomen maakunnista, sis samat: https://fi.wikipedia.org/wiki/Suomen_maakunnat
- Luettelo Suomen postinumeroista kunnittain: https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain


- http://tilastokeskus.fi/org/avoindata/paikkatietoaineistot.html

Tilastokeskus, Paavo:
- Tilastokeskus, Paavo kuvaus (postinumeroalueittainen avoin tieto): http://tilastokeskus.fi/tup/paavo/paavo_kuvaus_fi.pdf
- Tilastokeskuksen PX-Web-tietokannat (Paavo): http://pxnet2.stat.fi/PXWeb/pxweb/fi/Postinumeroalueittainen_avoin_tieto/
- PX-Web:ssä valittu 2019 -aineisto => 9. Kaikki tietoryhmät (päivitetty 22.1.2019) ja seuraavalla sivulla valittu kaikki postinumeroalueet ja kaikki tiedot => valinta yhteensä 314 808 taulukkosolua, lataus "puolipiste-eroteltu (otsikollinen)".  Tallennettu tiedostoon *paavo_9_koko.csv*
- from the saved file, first two rows (empty rows in the beginning of the file) were removed.
- coordinates used: **EUREF-FIN** coordinate system (**ETRS89-TM35FIN**)


- toinen lähde: http://spatialreference.org/ref/epsg/?search=finland&srtext=Search
- antaa EPSG:2393: KKJ / Finland Uniform Coordinate System
- tämäkin lähde auttoi: http://www.kolumbus.fi/eino.uikkanen/geodocsgb/ficoords.htm
- Test coordinates visually (are they in right place) here: https://suomenkartta.fi/karttakoordinaatit/


Protocol: Web Feature Service (WFS)
- In python lib OWSLib 0.17.1 here: https://geopython.github.io/OWSLib/
- returns GML files?


# Solution

- Using FourSquare to access venue data in selected areas
- Using Folium to show results on map
- Using Paavo data to get finnish postal codes and some information about each postal code area

...


# Step 0.1 - Workbook basic settings

Use the following variable to control certain aspects of this workbook.

In [1]:

# Paavo data filename.  First one (commented out) has the variable names in finnish,
# the latter one in English
#
# PAAVO_FILENAME = 'paavo_9_koko_sarkain_fixed.csv'
PAAVO_FILENAME = 'paavo_9_koko_en_tab.csv'


# This is the file where the downloaded Foursquare data is stored in CSV format.
FS_DATA_FILENAME = "FourSquare_downloaded_venues_new.csv"



# Change these values IF YOU DO WANT downloading to happen.  In most cases we should already have the venue data downloaded, so these
# should be False for not downloading.

FS_DOWNLOAD_ALL = False
FS_DOWNLOAD_SELECTED = False

# If you want to download the data, then also fill in these values.

FS_CLIENT_ID = '<YOUR_CLIENT_ID>' # your Foursquare ID
FS_CLIENT_SECRET = '<YOUR_CLIENT_SECRET>' # your Foursquare Secret



## Step 0.2 - Import libraries

In [2]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import os
import math
import requests # library to handle requests
import operator

#!conda install -c conda-forge folium=0.5.0 --yes
# OR
#!pip install folium
import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import preprocessing

#!pip install seaborn
import seaborn as sns

# Need to install pyproj first, if possible, check this out...
#!pip install pyproj
import pyproj

import matplotlib.pyplot as plt

print('Libraries imported.')

Libraries imported.


# Step 0.3 - Helper functions

These are gathered here to make the rest of the workbook a bit more readable.

In [56]:
# Helper functions for Paavo data Enhancement

# Following helper functions assume Paavo 'postal code' field value has
# Structure: 'NNNN <postal code area name> (<city name>)'
# Example: '00120 Punavuori (Helsinki )'

# Return the postal code number from Paavo 'postal code' field value
def get_pc_number(pc):
    return pc[:5].strip()

# Return the city name from Paavo 'postal code' field value
def get_city_name(pc):
    loc = pc.find('(')
    return pc[loc:].strip('() ')

# Return the postal code area name from Paavo 'postal code' field value
def get_pc_area_name(pc):
    loc = pc.find('(')
    return pc[5:loc].strip()
    
print("get_pc_number() defined.")
print("get_city_name() defined.")
print("get_pc_area_name() defined.")


# Generic helper functions

# Helper.  Whole is a list, in which we want to sort elements of top to the beginning.
# Parameters 'top' and 'whole' are both lists.  Every element of 'top' must be in 'whole'.
# Returns a new list, does not modify input parameters.
def list_order_to_top(top, whole):
    # Check that all elements in top are also in whole
    for t in top:
        if t not in whole:
            raise Exception("joo")
    rest = []
    for w in whole:
        if w not in top:
            rest = rest + [w]
    return top + rest


# Find out how many of each value there are in a list.
# Takes in a list and returns a dictionary, whose keys are list values, and values are # of occurrences on list
def find_uniques_with_counts(l, key_counts={}):
    for list_value in l:
        list_value = str(list_value)
        if list_value in key_counts.keys():
            key_counts[list_value] = key_counts[list_value] + 1
        else:
            key_counts[list_value] = 1
    return key_counts


# Pretty print the uniques with their counts.  Takes in the output dictionary
# that is returned from find_uniques_with_counts
def print_uniques_with_counts_dict(key_counts, print_threshold=0, total_values=0):
    for i in (range(len(key_counts.keys()))):
        max_key = max(key_counts.items(), key=operator.itemgetter(1))[0]
        if total_values == 0:
            total_percentage = ""
        else:
            total_percentage = int(100*(key_counts[max_key]/total_values))
        if key_counts[max_key] > print_threshold:
            print("{:>25} -- {:3} ({:2}%)".format(max_key, key_counts[max_key], total_percentage))
        key_counts.pop(max_key, None)


print("list_order_to_top() defined.")
print("find_uniques_with_counts() defined.")
print("print_uniques_with_counts_dict() defined.")


# Helper functions to analyze / report data values.

# Helper function to do the filtering and report as it goes.
def clean_paavo(df, col_index):
    col_name = df.columns[col_index]
    print("Cleaning away postal codes which have less than 30 inhabitants in column\n", col_name)
    count_before = df.shape[0]
    filtered_df = df[df[col_name] >= 30]
    count_after = filtered_df.shape[0]
    print("\tPostal Codes cleaned away:   ", count_before - count_after)
    print("\tContinuing with:              ", count_after, "Postal Codes\n")
    return filtered_df

print("clean_paavo() defined.")


get_pc_number() defined.
get_city_name() defined.
get_pc_area_name() defined.
list_order_to_top() defined.
find_uniques_with_counts() defined.
print_uniques_with_counts_dict() defined.
clean_paavo() defined.


## Step 1 - Paavo data

### Step 1.1 - Load Paavo data

In [3]:

paavo_df = None

if os.path.isfile(PAAVO_FILENAME):

    paavo_df = pd.read_csv(PAAVO_FILENAME, sep='\t', encoding='iso-8859-1')
    print("Loaded Paavo data.\nFound {} rows and {} columns of data.".format(paavo_df.shape[0], paavo_df.shape[1]))

else:
    print("Did not find data file:", PAAVO_FILENAME)

# Show a sample of data to understand what we have
paavo_df.head()

Loaded Paavo data.
Found 3027 rows and 105 columns of data.


,Postal code area,X coordinate in metres,Y coordinate in metres,Surface area,"Inhabitants, total, 2017 (HE)","Females, 2017 (HE)","Males, 2017 (HE)","Average age of inhabitants, 2017 (HE)","0-2 years, 2017 (HE)","3-6 years, 2017 (HE)",...,"T Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use, 2016 (TP)","U Activities of extraterritorial organisations and bodies, 2016 (TP)","X Industry unknown, 2016 (TP)","Inhabitants, 2016 (PT)","Employed, 2016 (PT)","Unemployed, 2016 (PT)","Children aged 0 to 14, 2016 (PT)","Students, 2016 (PT)","Pensioners, 2016 (PT)","Others, 2016 (PT)"
0,Finland,429300,7084490,390813692400,5513130,2793999,2719131,42,160297,240994,...,656,343,59,5503297,2275679,355837,894178,407905,1389830,179868
1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,385114,6672391,2353278,18284,9613,8671,41,434,521,...,0,17,1,18035,10032,856,1812,1198,3326,811
2,00120 Punavuori (Helsinki ),385614,6671378,414010,7108,3751,3357,40,183,234,...,0,0,0,7055,3872,336,817,428,1242,360
3,00130 Kaartinkaupunki (Helsinki ),386228,6671492,428960,1508,772,736,41,34,48,...,0,12,0,1522,839,61,170,104,258,90
4,00140 Kaivopuisto - Ullanlinna (Helsinki ),386410,6670742,931841,7865,4277,3588,41,224,250,...,0,1,0,7934,4218,325,929,518,1519,425


### Step 1.2 - Paavo data, exploration

Here we explore the pure Paavo data features / columns as they are available from Tilastokeskus, that is, no fixes, cleaning or data manipulation on values.

Paavo data contains data in 105 columns for 3026 postal codes plus for the whole finland as well.  Below are the different datacolumns and their values for whole Finland, Helsinki downtown and a random rural area. The data is from years 2016 and 2017, depending on the data.

In [4]:
# Create a subset dataframe to inspect data.  In the transposed dataframe:
#    - Column 0 is for whole Finland,
#    - Column 1 is for postal code 00100 (Finlands Capital, center)
#    - Column 2600 is for postal code 89840 (very rural area)
#
paavo_fin_df = paavo_df.T[[0, 1, 2600]]
paavo_fin_df.columns = ["Whole Finland", paavo_df.iloc[1,0], paavo_df.iloc[2600,0]]


#### Step 1.2.1 - Paavo data exploration - population and age groups, 2017

In [5]:
paavo_fin_df[4:28]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Inhabitants, total, 2017 (HE)",5513130,18284,57
"Females, 2017 (HE)",2793999,9613,20
"Males, 2017 (HE)",2719131,8671,37
"Average age of inhabitants, 2017 (HE)",42,41,63
"0-2 years, 2017 (HE)",160297,434,0
"3-6 years, 2017 (HE)",240994,521,0
"7-12 years, 2017 (HE)",369950,711,0
"13-15 years, 2017 (HE)",177163,274,0
"16-17 years, 2017 (HE)",117857,185,0
"18-19 years, 2017 (HE)",120218,264,1


#### Step 1.2.2 - Paavo data exploration - Education, 2017

In [6]:
paavo_fin_df[28:35]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Aged 18 or over, total, 2017 (KO)",4446869,16159,57
"Basic level studies, 2017 (KO)",1112261,1996,30
"With education, total, 2017 (KO)",3334608,14163,27
"Matriculation examination, 2017 (KO)",303230,2618,1
"Vocational diploma, 2017 (KO)",2035528,2942,24
"Academic degree - Lower level university degree, 2017 (KO)",518969,2899,2
"Academic degree - Higher level university degree, 2017 (KO)",476881,5704,0


#### Step 1.2.3 - Paavo data exploration - Inhabitant income, 2016

In [7]:
paavo_fin_df[35:42]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Aged 18 or over, total, 2016 (HR)",4431392,15935,60
"Average income of inhabitants, 2016 (HR)",23812,38985,16166
"Median income of inhabitants, 2016 (HR)",20925,26642,14939
"Inhabintants belonging to the lowest income category, 2016 (HR)",886431,2856,26
"Inhabitants belonging to the middle income category, 2016 (HR)",2658687,6668,31
"Inhabintants belonging to the highest income category, 2016 (HR)",886274,6411,3
"Accumulated purchasing power of inhabitants, 2016 (HR)",105520349469,621218859,969978


#### Step 1.2.4 - Paavo data exploration - Households, 2017

In [8]:
paavo_fin_df[42:57]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Households, total, 2017 (TE)",2680077,10205,34
"Average size of households, 2017 (TE)",2.0,1.8,1.7
"Occupancy rate, 2017 (TE)",40.5,38.6,56.4
"Young single persons, 2017 (TE)",291052,2101,1
"Young couples without children, 2017 (TE)",115168,861,0
"Households with children, 2017 (TE)",570112,1326,0
"Households with small children, 2017 (TE)",142781,400,0
"Households with children under school age, 2017 (TE)",278849,715,0
"Households with school-age children, 2017 (TE)",263490,541,0
"Households with teenagers, 2017 (TE)",221106,373,0


#### Step 1.2.5 - Paavo data exploration - Household income, 2016

In [9]:
paavo_fin_df[57:64]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Households, total, 2016 (TR)",2654657,10042,36
"Average income of households, 2016 (TR)",39270,61679,26975
"Median income of households, 2016 (TR)",31824,38895,23598
"Households belonging to the lowest income category, 2016 (TR)",677223,1697,13
"Households belonging to the middle income category, 2016 (TR)",1500917,4123,22
"Households belonging to the highest income category, 2016 (TR)",476517,4222,1
"Accumulated purchasing power of households, 2016 (TR)",104247634221,619383515,971110


#### Step 1.2.6 - Paavo data exploration - Buildings, 2017

In [10]:
paavo_fin_df[64:72]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Free-time residences, 2017 (RA)",507200,0,103
"Buildings, total, 2017 (RA)",1523196,634,90
"Other buildings, 2017 (RA)",228770,326,14
"Residential buildings, 2017 (RA)",1294426,308,76
"Dwellings, 2017 (RA)",2946814,11884,48
"Average floor area, 2017 (RA)",80.1,65.9,97.7
"Dwellings in small houses, 2017 (RA)",1568029,2,48
"Dwellings in blocks of flats, 2017 (RA)",1378785,11882,0


#### Step 1.2.7 - Paavo data exploration - Jobs, 2016

In [11]:
paavo_fin_df[72:98]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Workplaces, 2016 (TP)",2094313,48470,16
"Primary production, 2016 (TP)",56104,104,0
"Processing, 2016 (TP)",461153,1805,0
"Services, 2016 (TP)",1576997,46560,16
"A Agriculture, forestry and fishing, 2016 (TP)",56104,104,0
"B Mining and quarrying, 2016 (TP)",5283,0,0
"C Manufacturing, 2016 (TP)",283209,752,0
"D Electricity, gas, steam and air conditioning supply, 2016 (TP)",11714,554,0
"E Water supply; sewerage, waste management and remediation activities, 2016 (TP)",10703,1,0
"F Construction, 2016 (TP)",150244,498,0


#### Step 1.2.8 - Paavo data exploration - Main type of activity, 2016

In [12]:
paavo_fin_df[98:]

,Whole Finland,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,89840 Ylä-Vuokki (Suomussalmi )
"Inhabitants, 2016 (PT)",5503297,18035,61
"Employed, 2016 (PT)",2275679,10032,13
"Unemployed, 2016 (PT)",355837,856,6
"Children aged 0 to 14, 2016 (PT)",894178,1812,0
"Students, 2016 (PT)",407905,1198,1
"Pensioners, 2016 (PT)",1389830,3326,40
"Others, 2016 (PT)",179868,811,1


### Step 1.3 - Clean data

Paavo contains ".." values (two dots) in certain columns for such postal code areas where there are less than 30 cases in total for the data section.  Here cleaning means removing such rows from data so that we have only numeric values in columns other than the first column. 

Also, drop the first row of data which contains totals for whole Finland.

But first, define some helper functions.

Check the distribution of Paavo data column dtypes (of the dataframe). Remember, there are 105 columns in total.

In [14]:
t1 = paavo_df.dtypes.tolist()
d = find_uniques_with_counts(t1)
print_uniques_with_counts_dict(d, 0, 105)

                   object --  91 (86%)
                    int64 --  14 (13%)


So, many columns are not numeric type.  Now clean the ".." dots away.

In [15]:
print("\nCleaning Paavo data, start with {} postal codes.\n".format(paavo_df.shape[0]))

paavo_filtered_df = paavo_df

# filter on population structure (age) having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 4)

# filter on education having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 28)

# filter on income having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 35)

# filter on households having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 42)

# filter on household income having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 57)

# filter on jobs having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 72)

# filter on main type of activity having less than 30 cases
paavo_filtered_df = clean_paavo(paavo_filtered_df, 98)



Cleaning Paavo data, start with 3027 postal codes.

Cleaning away postal codes which have less than 30 inhabitants in column
 Inhabitants, total, 2017 (HE)
	Postal Codes cleaned away:    75
	Continuing with:               2952 Postal Codes

Cleaning away postal codes which have less than 30 inhabitants in column
 Aged 18 or over, total, 2017 (KO)
	Postal Codes cleaned away:    11
	Continuing with:               2941 Postal Codes

Cleaning away postal codes which have less than 30 inhabitants in column
 Aged 18 or over, total, 2016 (HR)
	Postal Codes cleaned away:    4
	Continuing with:               2937 Postal Codes

Cleaning away postal codes which have less than 30 inhabitants in column
 Households, total, 2017 (TE)
	Postal Codes cleaned away:    107
	Continuing with:               2830 Postal Codes

Cleaning away postal codes which have less than 30 inhabitants in column
 Households, total, 2016 (TR)
	Postal Codes cleaned away:    1
	Continuing with:               2829 Postal Code

Then try to convert the rest of columns to some numeric type, preferrence order is int, int64 and float.

In [16]:
print("Ensure number values in columns from 3 to end.\n")
for i in range(3, len(paavo_filtered_df.columns)):
    colname = paavo_filtered_df.columns[i]
    # don't convert floats, as they would become ints and lose the decimal parts.
    if str(paavo_filtered_df.iloc[:,i].dtype) != "float64":
        try:
            paavo_filtered_df.iloc[:,i] = paavo_filtered_df.iloc[:,i].astype("int")
        except:
            try:
                paavo_filtered_df.iloc[:,i] = paavo_filtered_df.iloc[:,i].astype(np.int64)
            except:
                try:
                    paavo_filtered_df.iloc[:,i] = paavo_filtered_df.iloc[:,i].astype("float")
                except:
                    print("*** Failed to convert {}th column ({}) to number".format(i, colname))

print("Column data types after cleaning:\n")
                    
t1 = paavo_filtered_df.dtypes.tolist()
d = find_uniques_with_counts(t1)
print_uniques_with_counts_dict(d, 0, 105)

Ensure number values in columns from 3 to end.

Column data types after cleaning:

                    int32 --  97 (92%)
                    int64 --   4 ( 3%)
                  float64 --   3 ( 2%)
                   object --   1 ( 0%)


Drop Finland country level totals from further analysis and check the data head after cleaning.

In [17]:
paavo_filtered_df = paavo_filtered_df.iloc[1:,:]
paavo_filtered_df.reset_index(inplace=True)
paavo_filtered_df.drop(columns=["index"], inplace=True)
paavo_filtered_df.head()

,Postal code area,X coordinate in metres,Y coordinate in metres,Surface area,"Inhabitants, total, 2017 (HE)","Females, 2017 (HE)","Males, 2017 (HE)","Average age of inhabitants, 2017 (HE)","0-2 years, 2017 (HE)","3-6 years, 2017 (HE)",...,"T Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use, 2016 (TP)","U Activities of extraterritorial organisations and bodies, 2016 (TP)","X Industry unknown, 2016 (TP)","Inhabitants, 2016 (PT)","Employed, 2016 (PT)","Unemployed, 2016 (PT)","Children aged 0 to 14, 2016 (PT)","Students, 2016 (PT)","Pensioners, 2016 (PT)","Others, 2016 (PT)"
0,00100 Helsinki Keskusta - Etu-Töölö (Helsinki,385114,6672391,2353278,18284,9613,8671,41,434,521,...,0,17,1,18035,10032,856,1812,1198,3326,811
1,00120 Punavuori (Helsinki ),385614,6671378,414010,7108,3751,3357,40,183,234,...,0,0,0,7055,3872,336,817,428,1242,360
2,00130 Kaartinkaupunki (Helsinki ),386228,6671492,428960,1508,772,736,41,34,48,...,0,12,0,1522,839,61,170,104,258,90
3,00140 Kaivopuisto - Ullanlinna (Helsinki ),386410,6670742,931841,7865,4277,3588,41,224,250,...,0,1,0,7934,4218,325,929,518,1519,425
4,00150 Eira - Hernesaari (Helsinki ),385235,6670549,1367328,9496,5129,4367,40,250,282,...,7,8,0,9527,5433,536,942,564,1523,529


### Step 1.3 - Enhance the data

1. Latitude and Longitude values for coordinates in addition to the metric X and Y values (use pyproj for this)
2. Separate the postal code number (5 digits) into a column of its own
3. Separate the city name (in parenthes) into a column of its own
4. Separate the postal code area name (the rest, between number and city name) into a column of its own.


In [19]:
# store original columns for easier sorting of new columns to the beginning
orig_cols = paavo_filtered_df.columns.tolist()

# 1. Get the latitude and longitude values, based on X and Y meters.
p = pyproj.Proj(proj='utm',zone=35,ellps='WGS84') # use kwarg
x_list = paavo_filtered_df.iloc[:,1].tolist()
y_list = paavo_filtered_df.iloc[:,2].tolist()
lon_list, lat_list = p(x_list, y_list, inverse=True)

# Add latitude and longitude into data
paavo_filtered_df["PC Longitude"] = lon_list
paavo_filtered_df["PC Latitude"] = lat_list

# 2. Create postal code number into a new column
paavo_filtered_df["PC"] = paavo_filtered_df.iloc[:,0].str.slice(stop=5)

# 3. Create city name into a new column
paavo_filtered_df["City"] = paavo_filtered_df.iloc[:,0].map(get_city_name)

# 4. change old postal code column into one containing only the name
postal_code_column_name = paavo_filtered_df.columns[0]
paavo_filtered_df[postal_code_column_name] = paavo_filtered_df[postal_code_column_name].map(get_pc_area_name)


# Sort the columns so that the new columns are on the left side (easier to see)
new_cols = ["PC", postal_code_column_name, "City", "PC Longitude", "PC Latitude"]
paavo_filtered_df = paavo_filtered_df[new_cols + orig_cols[1:]]
paavo_filtered_df.rename(columns={postal_code_column_name: "PC Name"}, inplace=True)

paavo_filtered_df.head()

,PC,PC Name,City,PC Longitude,PC Latitude,X coordinate in metres,Y coordinate in metres,Surface area,"Inhabitants, total, 2017 (HE)","Females, 2017 (HE)",...,"T Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use, 2016 (TP)","U Activities of extraterritorial organisations and bodies, 2016 (TP)","X Industry unknown, 2016 (TP)","Inhabitants, 2016 (PT)","Employed, 2016 (PT)","Unemployed, 2016 (PT)","Children aged 0 to 14, 2016 (PT)","Students, 2016 (PT)","Pensioners, 2016 (PT)","Others, 2016 (PT)"
0,00100,Helsinki Keskusta - Etu-Töölö,Helsinki,24.929290,60.172207,385114,6672391,2353278,18284,9613,...,0,17,1,18035,10032,856,1812,1198,3326,811
1,00120,Punavuori,Helsinki,24.938865,60.163257,385614,6671378,414010,7108,3751,...,0,0,0,7055,3872,336,817,428,1242,360
2,00130,Kaartinkaupunki,Helsinki,24.949856,60.164452,386228,6671492,428960,1508,772,...,0,12,0,1522,839,61,170,104,258,90
3,00140,Kaivopuisto - Ullanlinna,Helsinki,24.953552,60.157772,386410,6670742,931841,7865,4277,...,0,1,0,7934,4218,325,929,518,1519,425
4,00150,Eira - Hernesaari,Helsinki,24.932508,60.155712,385235,6670549,1367328,9496,5129,...,7,8,0,9527,5433,536,942,564,1523,529


Finally, create a standardized version of the Paavo data.  This is used later in analysis / clustering.

In [20]:
# Create a standardized version of Paavo data.  Don't include unnecessary columns

paavo_stan_cols = paavo_filtered_df.columns.tolist()
paavo_stan_cols = paavo_stan_cols[6:]
paavo_stan_df = paavo_filtered_df[paavo_stan_cols].astype('float64') # type conversions removes a warning
paavo_stan_np = preprocessing.StandardScaler().fit_transform(paavo_stan_df)
paavo_stan_df = pd.DataFrame(paavo_stan_np)
paavo_stan_df["PC"] = paavo_filtered_df["PC"]
paavo_stan_df.head()

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,PC
0,-1.100272,-0.452328,4.776863,4.838401,4.697720,-0.568997,3.407284,2.724038,2.445841,1.874344,...,3.253556,5.478789,4.745716,6.251701,2.915840,2.597545,3.406430,3.349305,6.101162,00100
1,-1.105567,-0.462058,1.389098,1.431900,1.338992,-0.765921,1.029136,0.815641,0.658872,0.484515,...,-0.031457,-0.149592,1.387119,1.963403,0.734109,0.750987,0.810015,0.759059,2.356837,00120
2,-1.104971,-0.461983,-0.308423,-0.299244,-0.317618,-0.568997,-0.382594,-0.421160,-0.476645,-0.497112,...,2.287376,-0.149592,-0.305332,-0.148026,-0.419691,-0.449740,-0.282503,-0.463974,0.115223,00130
3,-1.108892,-0.459460,1.618566,1.737567,1.484996,-0.568997,1.417598,0.922032,0.903167,0.785807,...,0.161779,-0.149592,1.655991,2.204272,0.687957,0.958841,1.113492,1.103348,2.896484,00140
4,-1.109901,-0.457275,2.112969,2.232678,1.977365,-0.765921,1.663940,1.134815,0.853403,0.397043,...,1.514431,-0.149592,2.143262,3.050097,1.573236,0.982967,1.268602,1.108320,3.759921,00150


## Step 2 - Foursquare venue data

### Step 2.1 - Download Foursquare data to a local CSV file (just once!)

This section contains code to download the venue data from Foursquare to a local CSV file.  Doing this is not necessary, if you already have the data in a downloaded CSV file.

If you want to redownload the data change the variable XXX to XXX and remove the target file if it exists.

Define more FS\_ parameters and helper functions so that we can easily get venue data for those locations that we choose.

In [21]:

# FourSquare call parameters
FS_VERSION = '20180604'
FS_LIMIT = 50 # This is maximum

venue_data_columns = [
    'PC', 
    'PC Latitude', 
    'PC Longitude',
    'Venue Id',
    'Venue', 
    'Venue Latitude', 
    'Venue Longitude', 
    'Venue Category']


# getNearbyVenues accesses Foursquare's explore -endpoint to download venue data, and returns a dataframe.
# Helper function from 2nd lab of week 3
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=500, section=False):
    
    venues_list=[]
    for pc, lat, lng in zip(postal_codes, latitudes, longitudes):
        #print(pc)

        # create the API request URL
        url = ""
        if section:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
                FS_CLIENT_ID, FS_CLIENT_SECRET, FS_VERSION, lat, lng, radius, section, FS_LIMIT)
        else:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                FS_CLIENT_ID, FS_CLIENT_SECRET, FS_VERSION, lat, lng, radius, FS_LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                pc, lat, lng,
                v['venue']['id'],
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']
            ) for v in results])
        except:
            print("SOMETHING WENT WRONG DOWNLOADING {} FROM FourSquare".format(pc))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if nearby_venues.shape[0] == 0:
        nearby_venues = pd.DataFrame(columns=venue_data_columns)
    else:
        nearby_venues.columns = venue_data_columns
    
    return(nearby_venues)


print("getNearbyVenues() defined")


# For each postal code in the input paavo_df, check if it already has some downloaded venue data in fs_venue_df.
# If not, then download venue data, otherwhise skip to next postal code in paavo_df.  Update the downloaded results
# directly to the fs_venue_df.
#
# Input paavo_df can be all of the paavo data or any subset of selected rows from it - this update focuses only on the
# postal codes included in the input paavo_df.
#
# Note that as the result limit from Foursquare is 50 venues.  If the returned result set indeed contains that maximum
# amount of venues, then we seek for more venue data by doing additional but more focused downloads to food, coffee and
# drinks categories. 

def update_fs_venues(paavo_df):
    global fs_venue_df
    work_df = paavo_df
    print("Filter results in {} postal codes. Starting...".format(work_df.shape[0]))

    # Loop through each postal code, check if there are already venues for it in fs_venue_df.
    # If yes, skip to next postal code, if not then download data and add it to fs_venue_df.
    for pc in work_df["PC"].tolist():
        #print(pc, type(pc))
        if fs_venue_df[fs_venue_df["PC"] == pc].shape[0] == 0:
            # No previous data, download and add
            print("downloading venues for {} from Foursquare.".format(pc), end="")
            
            pc_row_df = work_df[work_df["PC"] == pc]
            lon = pc_row_df.iloc[0,2]
            lat = pc_row_df.iloc[0,3]
            area = pc_row_df.iloc[0,6]
            r = int(math.sqrt(area))
            new_venues_df = getNearbyVenues([pc], [lat], [lon], r)
            fs_venue_df = pd.concat([fs_venue_df, new_venues_df])
            count = new_venues_df.shape[0]
            print("  Received {} venues.".format(count), end="")
            if count == 50:
                print(" More food, coffee and drinks?", end="")
                # Check for more 'food', 'drinks' and 'coffee' venues with the section info
                new_venues_df = getNearbyVenues([pc], [lat], [lon], r, 'food')
                fs_venue_df = pd.concat([fs_venue_df, new_venues_df])
                new_venues_df = getNearbyVenues([pc], [lat], [lon], r, 'coffee')
                fs_venue_df = pd.concat([fs_venue_df, new_venues_df])
                new_venues_df = getNearbyVenues([pc], [lat], [lon], r, 'drinks')
                fs_venue_df = pd.concat([fs_venue_df, new_venues_df])
            print(" Done.")
                
    print("Done.")


print("update_fs_venues() defined.")


# Save fs_venue_df to a local file and test it by reading it back and comparing the dataframe shapes.
#
# It is worthwhile to save the data every now and then, as it is
# a lot of downloading, takes a lot time, and something can fail.

def save_fs_to_disk():
    fs_venue_df.to_csv(FS_DATA_FILENAME, index=False)

    # Check via reading data back in
    test_read_df = pd.read_csv(FS_DATA_FILENAME)
    if fs_venue_df.shape == test_read_df.shape:
        print("File saved successfully with {} rows of data.".format(fs_venue_df.shape[0]))
        # Release this data from memory, it was just for testing.
        test_read_df = None
    else:
        print("Something is wrong, files do not match.")
        print("Data shape:", fs_venue_df.shape)
        print("File shape:", test_read_df.shape)
        test_read_df = None

print("save_fs_to_disk() defined.")

getNearbyVenues() defined
update_fs_venues() defined.
save_fs_to_disk() defined.


In [22]:


fs_venue_df = pd.DataFrame(columns=venue_data_columns)

if FS_DOWNLOAD_ALL or FS_DOWNLOAD_SELECTED:
    if os.path.isfile(FS_DATA_FILENAME):
        print("Found existing data, appending it.")
        fs_venue_df = pd.read_csv(FS_DATA_FILENAME, dtype={"PC": 'str'})
        print(fs_venue_df.shape)


# From experience we know that downloading all the data gives us over 30 000 rows of data into the file
if FS_DOWNLOAD_ALL and (fs_venue_df.shape[0] < 30000):
    # Run the download for all postal code areas and saving data to disk

    last_l_limit = 0

    # Loop downloading in small sections of 100 postal codes and save into file
    for u_limit in list(range(0, paavo_filtered_df.shape[0], 100)):
        l_limit = u_limit - 200
        l_limit = max(0, l_limit)
        last_l_limit = l_limit
        print("Working on range:", l_limit, u_limit)
        count_before = fs_venue_df.shape[0]
        update_fs_venues(paavo_filtered_df.iloc[l_limit:u_limit,:])
        count_after = fs_venue_df.shape[0]
        print("Venue data has now {} rows of data.".format(count_after))
        if count_after > count_before:
            save_fs_to_disk()

    # Make sure we got the last ones, too
    print("Working on final range from", l_limit, "to the end.")
    count_before = fs_venue_df.shape[0]
    update_fs_venues(paavo_filtered_df.iloc[last_l_limit:,:])
    count_after = fs_venue_df.shape[0]
    print("Venue data has now {} rows of data.".format(count_after))
    if count_after > count_before:
        save_fs_to_disk()
    
    # Finally, remove any duplicate rows.  There can be some for postal codes with over 50 venues,
    # because in such cases we additionally downloaded only restaurants, coffee shops and drinking places.
    #
    # If there are duplicate rows, they are identical so just take the first in such cases
    fs_venue_df = fs_venue_df.groupby(["PC", "Venue Id"]).first().reset_index()
    fs_venue_df = fs_venue_df[venue_data_columns]
    save_fs_to_disk()
    print("Downloading Foursquare venue data for all postal codes done.")

elif FS_DOWNLOAD_SELECTED:

    # Change this as you like
    selection = paavo_filtered_df.iloc[0:100,:]

    update_fs_venues(selection)
    fs_venue_df = fs_venue_df.groupby(["PC", "Venue Id"]).first().reset_index()
    fs_venue_df = fs_venue_df[venue_data_columns]
    save_fs_to_disk()
    print("Downloading Foursquare venue data for selected postal codes done.")

else:
    print("Not downloading, FS_DOWNLOAD_ALL and FS_DOWNLOAD_SELECTED are both False.")


Not downloading, FS_DOWNLOAD_ALL and FS_DOWNLOAD_SELECTED are both False.


### Step 2.2 - Load in the downloaded Foursquare data (CSV)

In [23]:

fs_venue_df = None

if os.path.isfile(FS_DATA_FILENAME):
    # load from file
    print("Reading venues from file")
    fs_venue_df = pd.read_csv(FS_DATA_FILENAME, dtype={"PC": 'str'})
else:
    raise Exception("File {} not found.".format(FS_DATA_FILENAME))

print(fs_venue_df.shape)
fs_venue_df.head()


Reading venues from file
(36690, 8)


,PC,PC Latitude,PC Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,00100,60.172207,24.92929,4adcdb1ff964a5208b5f21e3,Konditoria Café Briossi,60.167320,24.938287,Bakery
1,00100,60.172207,24.92929,4adcdb1ff964a5208d5f21e3,Fazer Café,60.168481,24.947506,Café
2,00100,60.172207,24.92929,4adcdb1ff964a520a65f21e3,Café Strindberg,60.167690,24.946243,Café
3,00100,60.172207,24.92929,4adcdb20f964a520ce5f21e3,KuuKuu,60.175400,24.925190,Scandinavian Restaurant
4,00100,60.172207,24.92929,4adcdb20f964a520cf5f21e3,St. Urho's Pub,60.173970,24.931500,Beer Bar


# Step 3 - Merge Paavo and venue data

Combine the separate datasets together for further analysis.  Do this by concentrating on the venue categories.  Create a column for each venue category, and then calculate sums of venues for each postal code area.  Finally merge these venue categories to Paavo data.

Start with the _onehot_ encoding with Pandas' *get_dummies* -method.

## Step 3 - Analysis of data

Analyze what we have and also cluster the data

### Step 3.1 - Prepare for analysis: onehot


One hot -encoding and additionally, the top 10 most common types of venues for each postal code area.

One hot -encoding: create a column for each kind of venue that we have for the neighborhoods (onehot encoding). Then summarize them by the neighborhood.  And finally find the 10 most common of the for each neighborhood.

In [59]:
print("So we have {} (rows, columns) of venue data for Finland".format(fs_venue_df.shape))
print('There are {} uniques categories.'.format(len(fs_venue_df['Venue Category'].unique())))

# one hot encoding
pc_venues_onehot_df = pd.get_dummies(fs_venue_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pc_venues_onehot_df['PC'] = fs_venue_df['PC']

pc_venues_onehot_df = pc_venues_onehot_df[list_order_to_top(['PC'], pc_venues_onehot_df.columns.tolist())]

print("Running all venue category types into columns of their own results in {} rows and columns.".format(pc_venues_onehot_df.shape))
pc_venues_onehot_df.head()

So we have (36690, 8) (rows, columns) of venue data for Finland
There are 461 uniques categories.
Running all venue category types into columns of their own results in (36690, 462) rows and columns.


,PC,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,00100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now *pc_venues_onehot_df* contains each venue as its own row.

Next get all postal code area venue offerings into one row by grouping them with PC, and keep their sum equal to one, meaning that if there are 2 airports, 2 trainstations and avideo store, both airport and trainstation columns get value 0.4 and video store would get 0.2 

In [60]:
pc_venues_onehot_grouped_df = pc_venues_onehot_df.groupby('PC').mean().reset_index()
print("PC venues grouped shape", pc_venues_onehot_grouped_df.shape)

# Merge the postal code -columns into the data so we can use it to filter data later
pc_venues_onehot_grouped_df = pc_venues_onehot_grouped_df.merge(paavo_filtered_df[["PC", "PC Name", "City"]], on="PC")
pc_venues_onehot_grouped_df = pc_venues_onehot_grouped_df[list_order_to_top(["PC", "PC Name", "City"], pc_venues_onehot_grouped_df.columns.tolist())] 
pc_venues_onehot_grouped_df.head()

PC venues grouped shape (2093, 462)


,PC,PC Name,City,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,00100,Helsinki Keskusta - Etu-Töölö,Helsinki,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.04,0.01,0.00,0.00,0.00,0.01,0.00,0.00
1,00120,Punavuori,Helsinki,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.01,0.00,0.00
2,00130,Kaartinkaupunki,Helsinki,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00140,Kaivopuisto - Ullanlinna,Helsinki,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00150,Eira - Hernesaari,Helsinki,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00


### Step 3.2 - 10 most common venues for each postal code area

In [61]:
#
# Helper function to focus attention on each neighborhoods most common venues
# Expect that unnecessary columns have been removed already
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

print("return_most_common_venues() defined.")


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
# top_columns is used later, too
top_columns = []
for ind in np.arange(num_top_venues):
    try:
        top_columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        top_columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted_df = pd.DataFrame(columns=["PC"] + top_columns)
pc_venues_sorted_df['PC'] = pc_venues_onehot_grouped_df['PC']

for ind in np.arange(pc_venues_onehot_grouped_df.shape[0]):
    pc_venues_sorted_df.iloc[ind, 1:] = return_most_common_venues(pc_venues_onehot_grouped_df.iloc[ind, 3:], num_top_venues)

pc_venues_sorted_df.head()

return_most_common_venues() defined.


,PC,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,00100,Café,Coffee Shop,Scandinavian Restaurant,Sushi Restaurant,Wine Bar,Bakery,Bar,Beer Bar,Pub,Cocktail Bar
1,00120,Café,Coffee Shop,Cocktail Bar,Wine Bar,Bar,Scandinavian Restaurant,Bakery,Sushi Restaurant,Park,Restaurant
2,00130,Coffee Shop,Café,Scandinavian Restaurant,Cocktail Bar,Bar,Hotel,Restaurant,Mediterranean Restaurant,Pizza Place,Beer Garden
3,00140,Café,Coffee Shop,Scandinavian Restaurant,Park,Bar,Bakery,Restaurant,Nightclub,Cocktail Bar,Italian Restaurant
4,00150,Café,Bar,Coffee Shop,Scandinavian Restaurant,Park,Beer Bar,Restaurant,Wine Bar,Bakery,Italian Restaurant


### Step 3.3 - Filter data

Paavo and downloaded FourSquare data covers whole of Finland.  Filter out the top 20 cities.  Plus Savonlinna :)

In [30]:

# Turn filters on / off with True / False -values. For now only one filter
FILTER_POSTAL_CODE_AREAS = True

# Top 20 cities in finland (by population) + Savonlinna
filter_cities = ["Helsinki", "Espoo", "Tampere", "Vantaa", "Oulu", "Turku", "Jyväskylä", "Lahti", "Kuopio", "Pori", "Kouvola", "Joensuu", "Lappeenranta",
                 "Vaasa", "Hämeenlinna", "Seinäjoki", "Rovaniemi", "Mikkeli", "Kotka", "Salo", "Savonlinna"]

print("\n\n")

filter = None
if FILTER_POSTAL_CODE_AREAS:
    filt = [False for n in pc_venues_onehot_grouped_df['PC']]
    for city in filter_cities:
        print("Filtering for city: '{}'".format(city))
        city_filter = pc_venues_onehot_grouped_df['City'] == city
        filt = [c or f for c, f in zip(filt, city_filter)]
    filter = pd.Series(data = filt)
else:
    # Effectively no filter, but fill it so that it will pass all data through.
    filter = pd.Series(data = [True for n in pc_venues_onehot_grouped_df['PC']])

filter_passed_through = len([x for x in filter if x])

# the all() method is kind of 'and' operation for the whole series value, it returns True only
# if all of the values in the series are True.  Thus it means there is no filtering.
if filter.all():
    print("No data filtering defined.\n")
else:
    print("Filtering in use, proceeding to clustering with", filter_passed_through, "cases out of", pc_venues_onehot_grouped_df.shape[0], "possible cases.\n")





Filtering for city: 'Helsinki'
Filtering for city: 'Espoo'
Filtering for city: 'Tampere'
Filtering for city: 'Vantaa'
Filtering for city: 'Oulu'
Filtering for city: 'Turku'
Filtering for city: 'Jyväskylä'
Filtering for city: 'Lahti'
Filtering for city: 'Kuopio'
Filtering for city: 'Pori'
Filtering for city: 'Kouvola'
Filtering for city: 'Joensuu'
Filtering for city: 'Lappeenranta'
Filtering for city: 'Vaasa'
Filtering for city: 'Hämeenlinna'
Filtering for city: 'Seinäjoki'
Filtering for city: 'Rovaniemi'
Filtering for city: 'Mikkeli'
Filtering for city: 'Kotka'
Filtering for city: 'Salo'
Filtering for city: 'Savonlinna'
Filtering in use, proceeding to clustering with 677 cases out of 2093 possible cases.



### Step 3.4 - Clustering

1. Put Paavo-data and FourSquare data together for Kclustering.
2. Run clustering algorithm a few times to see how big value K can have so that clusters are still big enough (smallest resulting cluster has at least five (5) elements in it).
3. Use the found K to create the clustering that we continue to explore

In [31]:
# 1. Gather data to use in CLUSTERING

paavo_fs_clustering_df = paavo_stan_df.merge(pc_venues_onehot_grouped_df[filter], on="PC")

print("Merging {} and {} columns together. Result has {} columns, it should be one less than the sum.".format(
    paavo_stan_df.shape[1], pc_venues_onehot_grouped_df[filter].shape[1], paavo_fs_clustering_df.shape[1]))
print("Merging {} and {} rows together. Result has {} columns, it should be the smaller of the two.".format(
    paavo_stan_df.shape[0], pc_venues_onehot_grouped_df[filter].shape[0], paavo_fs_clustering_df.shape[0]))

# the postal codes do no good when clustering, so don't include them
bare_clustering_data = paavo_fs_clustering_df.drop(['PC', 'PC Name', 'City'], axis=1, inplace=False)
bare_clustering_data.head()


Merging 104 and 464 columns together. Result has 567 columns, it should be one less than the sum.
Merging 2107 and 677 rows together. Result has 677 columns, it should be the smaller of the two.


,0,1,2,3,4,5,6,7,8,9,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,-1.100272,-0.452328,4.776863,4.838401,4.697720,-0.568997,3.407284,2.724038,2.445841,1.874344,...,0.0,0.0,0.037975,0.006329,0.0,0.0,0.0,0.012658,0.0,0.0
1,-1.105567,-0.462058,1.389098,1.431900,1.338992,-0.765921,1.029136,0.815641,0.658872,0.484515,...,0.0,0.0,0.046053,0.000000,0.0,0.0,0.0,0.006579,0.0,0.0
2,-1.104971,-0.461983,-0.308423,-0.299244,-0.317618,-0.568997,-0.382594,-0.421160,-0.476645,-0.497112,...,0.0,0.0,0.015748,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,-1.108892,-0.459460,1.618566,1.737567,1.484996,-0.568997,1.417598,0.922032,0.903167,0.785807,...,0.0,0.0,0.007812,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,-1.109901,-0.457275,2.112969,2.232678,1.977365,-0.765921,1.663940,1.134815,0.853403,0.397043,...,0.0,0.0,0.019355,0.000000,0.0,0.0,0.0,0.006452,0.0,0.0


In [32]:
# 2. Run the clustering algorithm with different values of K to see what size of clusters emerge

proposed_k = 1

for loop_kclusters in range(2, 15):
    clusters = KMeans(n_clusters=loop_kclusters, random_state=0).fit(bare_clustering_data).labels_
    clusters_counts_d = find_uniques_with_counts(clusters, {}) # returns a dictionary
    clusters_counts_l = list(clusters_counts_d.values())
    clusters_counts_l.sort(reverse=True)
    if clusters_counts_l[-1] >= 5:
        proposed_k = loop_kclusters
    print("cluster count: {:>2}, cluster sizes (sorted by size): {}".format(loop_kclusters, clusters_counts_l))

print("\nProposing k = {} for clusters count (smallest clusters size is still at least 5 elements)".format(proposed_k))


cluster count:  2, cluster sizes (sorted by size): [529, 148]
cluster count:  3, cluster sizes (sorted by size): [461, 176, 40]
cluster count:  4, cluster sizes (sorted by size): [362, 220, 86, 9]
cluster count:  5, cluster sizes (sorted by size): [345, 214, 93, 18, 7]
cluster count:  6, cluster sizes (sorted by size): [323, 199, 94, 37, 17, 7]
cluster count:  7, cluster sizes (sorted by size): [337, 190, 91, 27, 23, 8, 1]
cluster count:  8, cluster sizes (sorted by size): [290, 128, 124, 81, 27, 18, 8, 1]
cluster count:  9, cluster sizes (sorted by size): [303, 157, 84, 79, 27, 18, 7, 1, 1]
cluster count: 10, cluster sizes (sorted by size): [287, 191, 76, 69, 18, 17, 11, 6, 1, 1]
cluster count: 11, cluster sizes (sorted by size): [277, 189, 77, 74, 24, 16, 10, 6, 2, 1, 1]
cluster count: 12, cluster sizes (sorted by size): [278, 136, 80, 77, 53, 17, 16, 11, 6, 1, 1, 1]
cluster count: 13, cluster sizes (sorted by size): [278, 136, 80, 77, 53, 17, 16, 8, 6, 3, 1, 1, 1]
cluster count: 14,

In [33]:
# 3. Set number of clusters to 6, it seems to be best for top20 cities (after this number of very small cluster just increases)
kclusters = proposed_k

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bare_clustering_data)

# check cluster labels generated for each row in the dataframe
print("\n\nClustered", len(kmeans.labels_), "postal code areas into", kclusters, "clusters.\n")

# Show some distributions after clustering
k_df = pd.DataFrame(kmeans.labels_)
k_df.columns = ['ClusterLabel']
k_df["count"] = np.ones(len(kmeans.labels_))
kc_df = k_df.groupby("ClusterLabel").count()
kc_df



Clustered 677 postal code areas into 6 clusters.



,count
ClusterLabel,
0,323
1,17
2,37
3,199
4,7
5,94


### Step 3.5 - See clustering results

To check out what the clustering found, inspect the data.  But first put together a combined dataset of Paavo and FourSquare data, and add cluster info back into it.


In [34]:

# We applied filtering for the clustering data, thus we need to apply filtering to the
# resulting data
f_pc_venues_sorted_df = pc_venues_sorted_df[filter].reset_index()

# add clustering labels
f_pc_venues_sorted_df.insert(0, 'Cluster Label', kmeans.labels_)

# merge to add latitude/longitude for each neighborhood
paavo_fs_merged_df = pd.merge(paavo_filtered_df.rename(columns={"PC Number": "PC"}), f_pc_venues_sorted_df, on='PC')
paavo_fs_merged_df = paavo_fs_merged_df.drop('index', 1)
print("\n\nThe merged, filtered and clustered data *paavo_fs_merged_df* contains", paavo_fs_merged_df.shape[0], "postal code areas and ",
      paavo_fs_merged_df.shape[1], " data columns.\n")

cols = paavo_fs_merged_df.columns
new_order_cols = list_order_to_top(["PC", "PC Name", "City", "Cluster Label"] + top_columns, cols)
paavo_fs_merged_df = paavo_fs_merged_df[new_order_cols]
paavo_fs_merged_df.head(10) # check the last columns!



The merged, filtered and clustered data *paavo_fs_merged_df* contains 677 postal code areas and  120  data columns.



,PC,PC Name,City,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,"T Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use, 2016 (TP)","U Activities of extraterritorial organisations and bodies, 2016 (TP)","X Industry unknown, 2016 (TP)","Inhabitants, 2016 (PT)","Employed, 2016 (PT)","Unemployed, 2016 (PT)","Children aged 0 to 14, 2016 (PT)","Students, 2016 (PT)","Pensioners, 2016 (PT)","Others, 2016 (PT)"
0,00100,Helsinki Keskusta - Etu-Töölö,Helsinki,4,Café,Coffee Shop,Scandinavian Restaurant,Sushi Restaurant,Wine Bar,Bakery,...,0,17,1,18035,10032,856,1812,1198,3326,811
1,00120,Punavuori,Helsinki,2,Café,Coffee Shop,Cocktail Bar,Wine Bar,Bar,Scandinavian Restaurant,...,0,0,0,7055,3872,336,817,428,1242,360
2,00130,Kaartinkaupunki,Helsinki,2,Coffee Shop,Café,Scandinavian Restaurant,Cocktail Bar,Bar,Hotel,...,0,12,0,1522,839,61,170,104,258,90
3,00140,Kaivopuisto - Ullanlinna,Helsinki,5,Café,Coffee Shop,Scandinavian Restaurant,Park,Bar,Bakery,...,0,1,0,7934,4218,325,929,518,1519,425
4,00150,Eira - Hernesaari,Helsinki,2,Café,Bar,Coffee Shop,Scandinavian Restaurant,Park,Beer Bar,...,7,8,0,9527,5433,536,942,564,1523,529
5,00160,Katajanokka,Helsinki,3,Café,Bar,Scandinavian Restaurant,Restaurant,Boat or Ferry,Park,...,0,0,0,4448,2019,192,602,290,1148,197
6,00170,Kruununhaka,Helsinki,2,Café,Coffee Shop,Scandinavian Restaurant,Bar,Beer Bar,Hotel Bar,...,0,234,0,7427,4099,345,892,507,1297,287
7,00180,Kamppi - Ruoholahti,Helsinki,4,Café,Coffee Shop,Bar,Scandinavian Restaurant,Beer Bar,Pub,...,0,0,0,13555,7475,768,1618,859,2151,684
8,00190,Suomenlinna,Helsinki,0,Café,History Museum,Park,Boat or Ferry,Scenic Lookout,Island,...,0,9,0,748,371,44,173,42,89,29
9,00200,Lauttasaari,Helsinki,1,Bar,Café,Restaurant,Pizza Place,Grocery Store,Scandinavian Restaurant,...,0,0,0,15183,8094,714,2130,916,2810,519


#### Step 3.5.1 - Analysis 1 "Most common venue categories in clusters postal code areas"

In [35]:
import operator

threshold_percentage = 10
print("For each cluster, for each postal code area in that cluster, consider those venue categories that are mentioned in 1st, 2nd or 3rd most common ones.")
print("For each cluster, show categories that make it into the top 3 in most of the postal code areas.  For example, if a 'cafe' makes it into the top 3 for")
print("every postal code area in the cluster, then category 'cafe' should have 100% appearance in the analysis below.  Threshold for showing is", threshold_percentage, "%.")
for c_id in range(kclusters):
    clust = paavo_fs_merged_df[paavo_fs_merged_df["Cluster Label"] == c_id]
    cluster_size = clust.shape[0]
    cluster_threshold = (threshold_percentage / 100) * clust.shape[0]
    print("\nCluster {} has {} rows.".format(c_id, cluster_size))

    cat_counts = {}
    for pos in range(4,7):
        for category in clust.iloc[:,pos].tolist():
            if category in cat_counts.keys():
                cat_counts[category] = cat_counts[category] + 1
            else:
                cat_counts[category] = 1

    for i in (range(len(cat_counts.keys()))):
        key = max(cat_counts.items(), key=operator.itemgetter(1))[0]
        if cat_counts[key] > cluster_threshold:
            print("{:>25} -- {:3} ({:2}%)".format(key, cat_counts[key], int(100*(cat_counts[key]/cluster_size))))
        cat_counts.pop(key, None)

#clust.groupby("1st Most Common Venue").count()
#clust.head()


For each cluster, for each postal code area in that cluster, consider those venue categories that are mentioned in 1st, 2nd or 3rd most common ones.
For each cluster, show categories that make it into the top 3 in most of the postal code areas.  For example, if a 'cafe' makes it into the top 3 for
every postal code area in the cluster, then category 'cafe' should have 100% appearance in the analysis below.  Threshold for showing is 10 %.

Cluster 0 has 323 rows.
            Grocery Store --  91 (28%)
              Supermarket --  62 (19%)
                     Café --  46 (14%)
              Pizza Place --  39 (12%)
                    Hotel --  34 (10%)

Cluster 1 has 17 rows.
              Pizza Place --  10 (58%)
                     Café --   9 (52%)
                      Bar --   8 (47%)
               Restaurant --   4 (23%)
     Fast Food Restaurant --   4 (23%)
            Grocery Store --   3 (17%)
                 Bus Stop --   2 (11%)
     Gym / Fitness Center --   2 (11%)
  

Observations about typical postal code areas in clusters:
- clusters 0 and 1 are big and their typical top venues are about shopping and grocery stores.  Services like cafes or some sort of restaurant are on the list, but clearly behind the shopping super markets.
- clusters 2 and 3 are quite small and kind of marginal in this examination.  However, they seem to be quite clearly cafe and restaurant services oriented clusters.
- cluster 4 is kind of balanced between food/cafe services and supermarkets / grocery stores.
- cluster 5 is the most clearly cafe and restaurant services oriented cluster of all.



#### Step 3.5.2 - Analysis 2: compare average (over postal code areas) numbers for each cluster

1. Use the previous "one hot" format of each venue category, and sum up venues in each category in each postal code area.
2. Create an additional summary (column) of cafe activity in the postal code areas.
3. Combine together all of Paavo data and this one hot summary of FourSquare venues.  NOTE this creates a dataframe of 584 columns (onehot encoding creates a lot of columns)
4. we are ready to show (and explore) the average number (per postal code area) of each feature **WITHIN** the cluster.  Purpose is to see if clusters differ from each other in some easily noticeable way.


In [36]:
# 1. revisit the onehot data, this time use 'sum' to summarize the data by postal code area

pc_venues_onehot_grouped_s_df = pc_venues_onehot_df.groupby('PC').sum().reset_index()
print("PC venues grouped (count) shape", pc_venues_onehot_grouped_s_df.shape)

pc_venues_onehot_grouped_s_df = pc_venues_onehot_grouped_s_df.merge(paavo_filtered_df[["PC", "PC Name", "City"]], on="PC")
pc_venues_onehot_grouped_s_df = pc_venues_onehot_grouped_s_df[list_order_to_top(["PC", "PC Name", "City"], pc_venues_onehot_grouped_s_df.columns.tolist())] 
pc_venues_onehot_grouped_s_df.head()


PC venues grouped (count) shape (2093, 462)


,PC,PC Name,City,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,00100,Helsinki Keskusta - Etu-Töölö,Helsinki,0,0,0,0,0,0,0,...,0,0,6,1,0,0,0,2,0,0
1,00120,Punavuori,Helsinki,0,0,0,0,0,0,0,...,0,0,7,0,0,0,0,1,0,0
2,00130,Kaartinkaupunki,Helsinki,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
3,00140,Kaivopuisto - Ullanlinna,Helsinki,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,00150,Eira - Hernesaari,Helsinki,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,1,0,0


In [37]:
# 2. create one additional column to summarize 'cafe' activity in the area

# vcf = venue category filter
vcf_columns = []
look_for_these = ["cafe", "coffee"]
for c in pc_venues_onehot_grouped_s_df.columns:
    for l in look_for_these:
        if l in c.lower():
            vcf_columns = vcf_columns + [c]

# Create summary column, initialize with zeros, and then loop the vcf_columns to sum it up
pc_venues_onehot_grouped_s_df["Cafe Summary"] = np.zeros(pc_venues_onehot_grouped_s_df.shape[0])
for c in vcf_columns:
    pc_venues_onehot_grouped_s_df["Cafe Summary"] = pc_venues_onehot_grouped_s_df["Cafe Summary"] + pc_venues_onehot_grouped_s_df[c]

pc_venues_onehot_grouped_s_df["Cafe Summary"] = pc_venues_onehot_grouped_s_df["Cafe Summary"].astype('int')
print("\nCreated 'Cafe Summary' column by summing up following columns:", vcf_columns, "\n")

vcf_columns = ["PC"] + vcf_columns + ["Cafe Summary"]
pc_venues_onehot_grouped_s_df[vcf_columns].head()


Created 'Cafe Summary' column by summing up following columns: ['Cafeteria', 'Coffee Shop', 'College Cafeteria'] 



,PC,Cafeteria,Coffee Shop,College Cafeteria,Cafe Summary
0,00100,0,17,0,17
1,00120,0,17,1,18
2,00130,0,16,0,16
3,00140,0,12,0,12
4,00150,1,13,1,15


In [38]:
# 2.1 create one additional column to summarize 'restaurant' activity in the area

# vcf = venue category filter
vcf_columns = []
look_for_these = ["pizza", 'restaurant', 'blini', 'breakfast', 'buffet', 'burger', 'burrito', 'diner', 'food truck', 'fried chicken', 'noodle house', 'sandwich'
                 'steakhouse', 'taco', 'wings joint']
for c in pc_venues_onehot_grouped_s_df.columns:
    for l in look_for_these:
        if l in c.lower():
            vcf_columns = vcf_columns + [c]

# Create summary column, initialize with zeros, and then loop the vcf_columns to sum it up
pc_venues_onehot_grouped_s_df["Restaurant Summary"] = np.zeros(pc_venues_onehot_grouped_s_df.shape[0])
for c in vcf_columns:
    pc_venues_onehot_grouped_s_df["Restaurant Summary"] = pc_venues_onehot_grouped_s_df["Restaurant Summary"] + pc_venues_onehot_grouped_s_df[c]

pc_venues_onehot_grouped_s_df["Restaurant Summary"] = pc_venues_onehot_grouped_s_df["Restaurant Summary"].astype('int')
print("\nCreated 'Restaurant Summary' column by summing up following columns:", vcf_columns, "\n")

vcf_columns = ["PC", "Restaurant Summary"]
pc_venues_onehot_grouped_s_df[vcf_columns].head()


Created 'Restaurant Summary' column by summing up following columns: ['Afghan Restaurant', 'African Restaurant', 'American Restaurant', 'Asian Restaurant', 'Australian Restaurant', 'Austrian Restaurant', 'Bed & Breakfast', 'Belgian Restaurant', 'Blini House', 'Brazilian Restaurant', 'Breakfast Spot', 'Buffet', 'Burger Joint', 'Burrito Place', 'Cambodian Restaurant', 'Caucasian Restaurant', 'Chinese Restaurant', 'Comfort Food Restaurant', 'Dim Sum Restaurant', 'Diner', 'Doner Restaurant', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Ethiopian Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'Food Truck', 'French Restaurant', 'Fried Chicken Joint', 'German Restaurant', 'Greek Restaurant', 'Halal Restaurant', 'Hawaiian Restaurant', 'Himalayan Restaurant', 'Hungarian Restaurant', 'Indian Restaurant', 'Indonesian Restaurant', 'Italian Restaurant', 'Japanese Restaurant', 'Kebab Restaurant', 'Korean Restaurant', 'Kurdish Restaurant', 'Lebanese Resta

,PC,Restaurant Summary
0,00100,41
1,00120,43
2,00130,41
3,00140,41
4,00150,39


In [39]:
# 3. merge...
paavo_fs_merged_df = pd.merge(paavo_fs_merged_df, pc_venues_onehot_grouped_s_df.drop(['PC Name', 'City'], axis=1, inplace=False), on="PC")
print("Shape of merged data:", paavo_fs_merged_df.shape)
# paavo_fs_merged_df.head()

Shape of merged data: (677, 583)


In [40]:
paavo_fs_merged_df.columns.tolist()[16:20]

['X coordinate in metres',
 'Y coordinate in metres',
 'Surface area',
 'Inhabitants, total, 2017 (HE)']

In [41]:
# 4. Show average numbers (over postal code areas) for each cluster

# Show floats in two decimals
# Note: this has global effect on all DataFrames!
pd.options.display.float_format = '{:,.2f}'.format

paavo_column_names = paavo_fs_merged_df.columns.tolist()

clus_df = paavo_fs_merged_df
clus_df = clus_df.groupby("Cluster Label").mean()
# in the following, columns 16 and 17 refer to X and Y coordinates of the postal code area.
clus_df = clus_df.drop(columns=["PC Longitude", "PC Latitude", paavo_column_names[16], paavo_column_names[17]])
# convert value of 'surface area' from square meters to square kilometers
clus_df[paavo_column_names[18]] = clus_df[paavo_column_names[18]] / 1000000
#clus_df = clus_df.astype("int")

#
# Change values in following columns into percentiges, which makes relative comparisons easier
#
clus_df.iloc[:, 26] = clus_df.iloc[:,26] / clus_df.iloc[:,25]
clus_df.iloc[:, 27] = clus_df.iloc[:,27] / clus_df.iloc[:,25]
clus_df.iloc[:, 28] = clus_df.iloc[:,28] / clus_df.iloc[:,25]
clus_df.iloc[:, 29] = clus_df.iloc[:,29] / clus_df.iloc[:,25]
clus_df.iloc[:, 30] = clus_df.iloc[:,30] / clus_df.iloc[:,25]
clus_df.iloc[:, 31] = clus_df.iloc[:,31] / clus_df.iloc[:,25]
clus_df.iloc[:, 35] = clus_df.iloc[:,35] / clus_df.iloc[:,32]
clus_df.iloc[:, 36] = clus_df.iloc[:,36] / clus_df.iloc[:,32]
clus_df.iloc[:, 37] = clus_df.iloc[:,37] / clus_df.iloc[:,32]
clus_df.iloc[:, 42] = clus_df.iloc[:,42] / clus_df.iloc[:,39]
clus_df.iloc[:, 43] = clus_df.iloc[:,43] / clus_df.iloc[:,39]
clus_df.iloc[:, 44] = clus_df.iloc[:,44] / clus_df.iloc[:,39]
clus_df.iloc[:, 45] = clus_df.iloc[:,45] / clus_df.iloc[:,39]
clus_df.iloc[:, 46] = clus_df.iloc[:,46] / clus_df.iloc[:,39]
clus_df.iloc[:, 47] = clus_df.iloc[:,47] / clus_df.iloc[:,39]
clus_df.iloc[:, 48] = clus_df.iloc[:,48] / clus_df.iloc[:,39]
clus_df.iloc[:, 49] = clus_df.iloc[:,49] / clus_df.iloc[:,39]
clus_df.iloc[:, 50] = clus_df.iloc[:,50] / clus_df.iloc[:,39]
clus_df.iloc[:, 51] = clus_df.iloc[:,51] / clus_df.iloc[:,39]
clus_df.iloc[:, 52] = clus_df.iloc[:,52] / clus_df.iloc[:,39]
clus_df.iloc[:, 53] = clus_df.iloc[:,53] / clus_df.iloc[:,39]
clus_df.iloc[:, 57] = clus_df.iloc[:,57] / clus_df.iloc[:,54]
clus_df.iloc[:, 58] = clus_df.iloc[:,58] / clus_df.iloc[:,54]
clus_df.iloc[:, 59] = clus_df.iloc[:,59] / clus_df.iloc[:,54]
clus_df.iloc[:, 70] = clus_df.iloc[:,70] / clus_df.iloc[:,69]
clus_df.iloc[:, 71] = clus_df.iloc[:,71] / clus_df.iloc[:,69]
clus_df.iloc[:, 72] = clus_df.iloc[:,72] / clus_df.iloc[:,69]
clus_df.iloc[:, 96] = clus_df.iloc[:,96] / clus_df.iloc[:,95]
clus_df.iloc[:, 97] = clus_df.iloc[:,97] / clus_df.iloc[:,95]
clus_df.iloc[:, 98] = clus_df.iloc[:,98] / clus_df.iloc[:,95]
clus_df.iloc[:, 99] = clus_df.iloc[:,99] / clus_df.iloc[:,95]
clus_df.iloc[:, 100] = clus_df.iloc[:,100] / clus_df.iloc[:,95]
clus_df.iloc[:, 101] = clus_df.iloc[:,101] / clus_df.iloc[:,95]


Below is data to explore.  Because of many aspects in Paavo data, this exploration is shown in many tables.

In [42]:
print("For each cluster, average data of its postal code areas. Columns 6-10 are percentiles of the amount in column 5.")
clus_df.iloc[:, [0,1,2,3,25,26,28,29,30,31]]


For each cluster, average data of its postal code areas. Columns 6-10 are percentiles of the amount in column 5.


,Surface area,"Inhabitants, total, 2017 (HE)","Females, 2017 (HE)","Males, 2017 (HE)","Aged 18 or over, total, 2017 (KO)","Basic level studies, 2017 (KO)","Matriculation examination, 2017 (KO)","Vocational diploma, 2017 (KO)","Academic degree - Lower level university degree, 2017 (KO)","Academic degree - Higher level university degree, 2017 (KO)"
Cluster Label,,,,,,,,,,
0,68.35,"1,065.44",528.94,536.50,837.86,0.23,0.05,0.50,0.12,0.10
1,7.89,"18,656.35","9,671.41","8,984.94","15,379.53",0.25,0.11,0.38,0.13,0.14
2,10.30,"8,304.46","4,442.22","3,862.24","7,366.95",0.20,0.13,0.36,0.15,0.16
3,36.07,"4,389.19","2,244.91","2,144.28","3,548.74",0.23,0.08,0.44,0.13,0.13
4,4.72,"18,099.43","9,604.86","8,494.57","16,482.00",0.16,0.17,0.29,0.17,0.20
5,19.89,"9,236.24","4,785.01","4,451.23","7,315.90",0.23,0.09,0.38,0.14,0.16


In [43]:
print("For each cluster, average data of its postal code areas. Columns 4-6 are percentiles of the amount in column 1.")
clus_df.iloc[:, 32:39]


For each cluster, average data of its postal code areas. Columns 4-6 are percentiles of the amount in column 1.


,"Aged 18 or over, total, 2016 (HR)","Average income of inhabitants, 2016 (HR)","Median income of inhabitants, 2016 (HR)","Inhabintants belonging to the lowest income category, 2016 (HR)","Inhabitants belonging to the middle income category, 2016 (HR)","Inhabintants belonging to the highest income category, 2016 (HR)","Accumulated purchasing power of inhabitants, 2016 (HR)"
Cluster Label,,,,,,,
0,836.01,"24,077.68","21,363.63",0.19,0.59,0.21,"20,856,991.58"
1,"15,164.00","23,688.12","21,134.12",0.21,0.59,0.19,"355,008,088.82"
2,"7,287.97","25,009.08","20,798.19",0.22,0.58,0.20,"177,064,174.51"
3,"3,522.98","24,311.61","21,590.13",0.20,0.59,0.20,"84,925,314.53"
4,"16,345.00","26,244.14","21,244.71",0.22,0.55,0.23,"425,237,865.00"
5,"7,240.43","25,796.12","22,803.34",0.19,0.57,0.24,"184,407,190.15"


In [44]:
print("For each cluster, average data of its postal code areas. Columns 4-15 are percentiles of the amount in column 1.")
clus_df.iloc[:, 39:54]

For each cluster, average data of its postal code areas. Columns 4-15 are percentiles of the amount in column 1.


,"Households, total, 2017 (TE)","Average size of households, 2017 (TE)","Occupancy rate, 2017 (TE)","Young single persons, 2017 (TE)","Young couples without children, 2017 (TE)","Households with children, 2017 (TE)","Households with small children, 2017 (TE)","Households with children under school age, 2017 (TE)","Households with school-age children, 2017 (TE)","Households with teenagers, 2017 (TE)","Adult households, 2017 (TE)","Pensioner households, 2017 (TE)","Households living in owner-occupied dwellings, 2017 (TE)","Households living in rented dwellings, 2017 (TE)","Households living in other dwellings, 2017 (TE)"
Cluster Label,,,,,,,,,,,,,,,
0,482.82,2.20,44.19,0.07,0.03,0.25,0.06,0.12,0.12,0.10,0.44,0.32,0.75,0.23,0.02
1,"9,825.65",1.88,35.48,0.15,0.07,0.19,0.06,0.10,0.08,0.07,0.55,0.26,0.45,0.53,0.02
2,"5,063.38",1.64,36.56,0.22,0.08,0.11,0.03,0.06,0.04,0.04,0.62,0.27,0.43,0.54,0.02
3,"2,204.96",2.02,39.15,0.13,0.05,0.21,0.05,0.10,0.10,0.08,0.51,0.28,0.59,0.40,0.02
4,"11,254.71",1.60,36.03,0.25,0.10,0.09,0.03,0.05,0.03,0.03,0.67,0.24,0.41,0.57,0.02
5,"4,529.61",2.06,36.35,0.12,0.05,0.24,0.06,0.12,0.11,0.08,0.52,0.25,0.53,0.45,0.02


In [45]:
print("For each cluster, average data of its postal code areas.")
clus_df.iloc[:, 54:57]

For each cluster, average data of its postal code areas.


,"Households, total, 2016 (TR)","Average income of households, 2016 (TR)","Median income of households, 2016 (TR)"
Cluster Label,,,
0,479.90,"42,734.98","36,926.39"
1,"9,638.94","37,205.00","30,065.94"
2,"4,966.51","37,328.35","28,392.84"
3,"2,174.19","40,381.02","33,912.67"
4,"11,075.71","38,984.86","28,938.00"
5,"4,457.51","42,542.05","35,052.87"


In [46]:
print("For each cluster, average data of its postal code areas.")
clus_df.iloc[:, 61:69]

For each cluster, average data of its postal code areas.


,"Free-time residences, 2017 (RA)","Buildings, total, 2017 (RA)","Other buildings, 2017 (RA)","Residential buildings, 2017 (RA)","Dwellings, 2017 (RA)","Average floor area, 2017 (RA)","Dwellings in small houses, 2017 (RA)","Dwellings in blocks of flats, 2017 (RA)"
Cluster Label,,,,,,,,
0,167.31,404.27,61.34,342.93,536.72,95.34,401.26,135.46
1,22.76,"1,561.41",181.41,"1,380.00","10,464.24",66.36,"2,081.71","8,382.53"
2,46.70,719.78,213.30,506.49,"5,640.43",59.46,555.95,"5,084.49"
3,80.49,878.97,110.03,768.95,"2,397.72",78.40,"1,048.35","1,349.37"
4,25.00,825.29,300.57,524.71,"12,920.86",56.86,500.29,"12,420.57"
5,57.52,"1,223.28",113.10,"1,110.18","4,837.54",74.39,"1,632.54","3,205.00"


In [47]:
print("For each cluster, average data of its postal code areas. Columns 2-4 are percentiles of the amount in column 1.")
clus_df.iloc[:, 69:73]

For each cluster, average data of its postal code areas. Columns 2-4 are percentiles of the amount in column 1.


,"Workplaces, 2016 (TP)","Primary production, 2016 (TP)","Processing, 2016 (TP)","Services, 2016 (TP)"
Cluster Label,,,,
0,563.76,0.02,0.22,0.76
1,"4,417.82",0.00,0.17,0.83
2,"9,457.49",0.00,0.13,0.87
3,"1,473.91",0.01,0.28,0.71
4,"24,132.86",0.00,0.09,0.91
5,"2,038.57",0.00,0.17,0.83


In [48]:
print("For each cluster, average data of its postal code areas. Columns 2-7 are percentiles of the amount in column 1.")
clus_df.iloc[:, 95:102]

For each cluster, average data of its postal code areas. Columns 2-7 are percentiles of the amount in column 1.


,"Inhabitants, 2016 (PT)","Employed, 2016 (PT)","Unemployed, 2016 (PT)","Children aged 0 to 14, 2016 (PT)","Students, 2016 (PT)","Pensioners, 2016 (PT)","Others, 2016 (PT)"
Cluster Label,,,,,,,
0,"1,065.68",0.41,0.07,0.18,0.07,0.25,0.03
1,"18,407.29",0.42,0.08,0.15,0.09,0.22,0.04
2,"8,219.78",0.45,0.07,0.10,0.09,0.25,0.04
3,"4,363.92",0.42,0.07,0.16,0.08,0.23,0.03
4,"17,920.43",0.51,0.07,0.08,0.10,0.22,0.03
5,"9,146.86",0.44,0.07,0.18,0.08,0.20,0.03


Observations:
- cluster 1 postal code areas are on average by far the largest, while their population is also the smallest.  Cluster 0 follows cluster 1 on this, but not so heavily.  Yet both are remarkably different from other clusters in these regards.  In the field of education balance, these clusters are clearly less academic and more hands on, education that prepares directly for some profession like cook, mechanic, constructions etc.
- Cluster 2 has on average the smallest postal code areas and most dense by population, and is on average the most educated, clearly geared towards academic education.  It has proportionally the least kids (age under 15) on average, but then again most studends by a slight margin. Cluster 2 postal code areas have also the most jobs on average, by a clear margin, and they are mostly service area professions.  Economically cluster 2 areas have highest average income.
- Cluster 3 follows cluster 2, but now in such extremes.  Postal code areas are still rather small, and these have on average the most population, but not so densily populated (almost twice the area / population when compared to cluster 2).  Proportion of kids in these postal code areas is average in comparison to other clusters.  These postal codes are average on the job amounts they offer.  Cluster 3 areas have lowest average income.
- Clusters 4 and 5 are between above clusters, but closer to clusters 2 and 3 than 0 and 1.  Cluster 5 postal code areas are on average a bit smaller than cluster 4 counterparts, and cluster 5 has a small edge in academic education (mainly in high school level, not university level).  What divides clusters 4 and 5 is that cluster 5 postal code areas offer twice more jobs than cluster 4 counterparts, but cluster 4 inhabitants have on average better income than inhabitants in cluster 5.

It would appear that cluster 2 is areas in big city centers, and cluster 3 is perhaps the neighborhood around these center / downtown areas.  And similarly cluster 5 would be average city central areas, while cluster 4 are the surrounding neighborhoods.  Clusters 0 and 1 are rather on the rural side of the scale.  Perhaps also in the smaller cities the well earning people like to live a bit outside the center, where as in the biggest cities the best earning people live in the very centres.

#### Step 3.5.3 - Analysis 3: Visual inspection of clusters on map

Each dot on the map marks a postal code area that was involved in analysis. Dot is located roughly in the center of the postal code area.

Cluster colors:
- cluster 0: blue
- cluster 1: brown
- cluster 2: black
- cluster 3: grey
- cluster 4: pink
- cluster 5: red

Zoom into the map to see in more detail!

In [49]:

my_color = "#050505"
#finland_latitude = 62.777754
finland_latitude = 65.0
finland_longitude = 26.199539
map_test = folium.Map(location=[finland_latitude, finland_longitude], zoom_start=5)

cluster_colors = ["#0505ff", "#cc8305", "#050505", "#707070", "#ff7575", "#ff0000"]
for i in range(paavo_fs_merged_df.shape[0]):
#for i in range(5):
    row = paavo_fs_merged_df.iloc[i,:]
#    print(row[0], row[1], row[2], row[14], row[15])
    
    # show postal code center location
    label = folium.Popup(row[0] + " " + row[1] + ", " + row[2] + ", cluster " + str(row[3]))
    my_color = cluster_colors[row[3]]
    folium.CircleMarker(
        [row[15], row[14]],
        radius=2,
        popup=label,
        color=my_color,
        fill_color=my_color,
        fill_opacity=0.8).add_to(map_test)

map_test

Observations:

Cluster 2 does indeed seem to be in the very center of biggest cities.  Then again cluster 5 is not just in smaller cities, but there are plenty cluster 5 areas in big cities as well.

Another observation is that these classifications and above attempt to understand them based on averages cannot be blindly trusted.  For example, one of Espoo's most wealthy inhabitan areas 'Westend' is classified to cluster 1.  It is likely to hold true for it that there are not so many services in the area, as it is mainly a neighborhood of big houses and estates, but clustering it together with actually dying rural areas is a sign that we need to watch out what we can deduce from average numbers.



#### Step 3.5.4 - Analysis 4: Compare citywise cluster percentages etc.

Check out how clusters are presented in each city.  That is, how the postal code areas in each city map to different clusters.

In [50]:
paavo_fs_merged_df.iloc[0:5, 18:20]

,Surface area,"Inhabitants, total, 2017 (HE)"
0,2353278,18284
1,414010,7108
2,428960,1508
3,931841,7865
4,1367328,9496


In [51]:
# use onehot encoding to be able to get cluster values into separate columns 
city_clusters_onehot_df = pd.get_dummies(paavo_fs_merged_df['Cluster Label'])
city_clusters_onehot_df['City'] = paavo_fs_merged_df['City']
city_clusters_onehot_df['Area (km2)'] = paavo_fs_merged_df.iloc[:, 18] / 1000000
city_clusters_onehot_df['Area (km2)'] = city_clusters_onehot_df['Area (km2)'].astype('int')
city_clusters_onehot_df['Population'] = paavo_fs_merged_df.iloc[:, 19].astype('int')
city_clusters_onehot_df['Cafe Summary'] = paavo_fs_merged_df['Cafe Summary'].astype('int')
city_clusters_onehot_df['Restaurant Summary'] = paavo_fs_merged_df['Restaurant Summary'].astype('int')

city_clusters_onehot_df['PC Area count'] = np.ones(city_clusters_onehot_df.shape[0]).astype('int')

city_clusters_onehot_df = city_clusters_onehot_df.groupby('City').sum()

# change cluster counts into percent values
for i in range(0, 6):
    None
    city_clusters_onehot_df[i] = 100 * (city_clusters_onehot_df[i] / city_clusters_onehot_df['PC Area count'])

city_clusters_onehot_df.sort_values(by=['Population'], ascending=False, inplace=True)
city_clusters_onehot_df["Biggest in Finland (population)"] = np.array(list(range(1, city_clusters_onehot_df.shape[0] + 1)))
city_clusters_onehot_df['Population by cafe'] = city_clusters_onehot_df['Population'] / city_clusters_onehot_df['Cafe Summary']
city_clusters_onehot_df['Population by cafe'] = city_clusters_onehot_df['Population by cafe'].astype('int')
city_clusters_onehot_df['Population by restaurant'] = city_clusters_onehot_df['Population'] / city_clusters_onehot_df['Restaurant Summary']
city_clusters_onehot_df['Population by restaurant'] = city_clusters_onehot_df['Population by restaurant'].astype('int')

city_clusters_onehot_df

,0,1,2,3,4,5,Area (km2),Population,Cafe Summary,Restaurant Summary,PC Area count,Biggest in Finland (population),Population by cafe,Population by restaurant
City,,,,,,,,,,,,,,
Helsinki,14.46,4.82,14.46,22.89,3.61,39.76,172,628760,328,1351,83,1,1916,465
Espoo,20.00,4.44,6.67,44.44,0.00,24.44,331,273029,137,457,45,2,1992,597
Tampere,32.35,8.82,11.76,26.47,2.94,17.65,735,223117,43,397,34,3,5188,562
Vantaa,31.43,8.57,2.86,31.43,0.00,25.71,226,215273,71,238,35,4,3032,904
Oulu,30.77,0.00,5.13,35.90,0.00,28.21,2592,199340,18,288,39,5,11074,692
Turku,19.23,11.54,7.69,38.46,3.85,19.23,279,188966,40,356,26,6,4724,530
Jyväskylä,37.50,3.12,3.12,43.75,3.12,9.38,1290,139297,37,196,32,7,3764,710
Kuopio,70.00,0.00,2.00,24.00,0.00,4.00,3707,115608,15,161,50,8,7707,718
Lahti,37.93,3.45,3.45,51.72,0.00,3.45,452,112303,29,187,29,9,3872,600


One observation here is that only eight (8) of the cities have at all cluster 2 and 3 areas.  And without the exception of Vaasa cities have all kinds of cluster areas present, at least for a small degree.  These cities are mainly the upper end of the top 20 cities in Finland (measured by population)


In [52]:
city_clusters_onehot_df[(city_clusters_onehot_df[2] > 0) | (city_clusters_onehot_df[3] > 0)]

,0,1,2,3,4,5,Area (km2),Population,Cafe Summary,Restaurant Summary,PC Area count,Biggest in Finland (population),Population by cafe,Population by restaurant
City,,,,,,,,,,,,,,
Helsinki,14.46,4.82,14.46,22.89,3.61,39.76,172,628760,328,1351,83,1,1916,465
Espoo,20.00,4.44,6.67,44.44,0.00,24.44,331,273029,137,457,45,2,1992,597
Tampere,32.35,8.82,11.76,26.47,2.94,17.65,735,223117,43,397,34,3,5188,562
Vantaa,31.43,8.57,2.86,31.43,0.00,25.71,226,215273,71,238,35,4,3032,904
Oulu,30.77,0.00,5.13,35.90,0.00,28.21,2592,199340,18,288,39,5,11074,692
Turku,19.23,11.54,7.69,38.46,3.85,19.23,279,188966,40,356,26,6,4724,530
Jyväskylä,37.50,3.12,3.12,43.75,3.12,9.38,1290,139297,37,196,32,7,3764,710
Kuopio,70.00,0.00,2.00,24.00,0.00,4.00,3707,115608,15,161,50,8,7707,718
Lahti,37.93,3.45,3.45,51.72,0.00,3.45,452,112303,29,187,29,9,3872,600


Of the other cities, not all have cluster 4 areas.  These cities do have:

In [53]:
city_clusters_onehot_df[(city_clusters_onehot_df[2] == 0) & (city_clusters_onehot_df[3] == 0) & (city_clusters_onehot_df[4] > 0)]

,0,1,2,3,4,5,Area (km2),Population,Cafe Summary,Restaurant Summary,PC Area count,Biggest in Finland (population),Population by cafe,Population by restaurant
City,,,,,,,,,,,,,,


Cities left out of the two above 'groups' are following.  They are quite heavily cluster 1 oriented cities.  Also, they are mainly the lower end of the top 20 cities in Finland (measured by population)

In [54]:
city_clusters_onehot_df[(city_clusters_onehot_df[2] == 0) & (city_clusters_onehot_df[3] == 0) & (city_clusters_onehot_df[4] == 0)]

,0,1,2,3,4,5,Area (km2),Population,Cafe Summary,Restaurant Summary,PC Area count,Biggest in Finland (population),Population by cafe,Population by restaurant
City,,,,,,,,,,,,,,


In [ ]:

city_venues_df = pc_venues_onehot_df.groupby('PC').sum()
city_venues_df.head()
#pc_venues_onehot_df.head()


In [ ]:
paavo_fs_merged_df.head()

#### Step 3.5.5 - Analysis 5: find correlations between coffees and other data

FInd out if there is a linear correlation between coffees and some other data features.

In [ ]:
# run linear regression correlation between 'coffee sum' and other columns from column 17 onwards
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

print("Test linear regression accuracies between 'Cafe Summary' and other columns")
#y = np.array(paavo_fs_merged_df.iloc[:, 123]).reshape(-1, 1)
y = np.array(paavo_fs_merged_df["Cafe Summary"]).reshape(-1, 1)
test_scores = []
test_scores_i = 0        # helper index for debugging
for i in range(18, paavo_fs_merged_df.shape[1]):
    colname = paavo_fs_merged_df.columns.tolist()[i]
    if colname != "Cafe Summary":
        X = np.array(paavo_fs_merged_df.iloc[:, i]).reshape(-1, 1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

        reg = LinearRegression()
        reg = reg.fit(X_train, y_train)
        yhat = reg.predict(X_test)

        test_scores.append([test_scores_i, i, colname, reg.score(X_train, y_train), reg.score(X_test, y_test)])
        test_scores_i += 1
    
print("\nTop 20 best correlating columns: (descending order in training accuracy score)\n")
top_20 = []
for j in range(20):
    max_key = max(test_scores, key=operator.itemgetter(3))[0]
    top_20.append(test_scores[max_key])
    test_scores[max_key] = [0, 0, 0, 0, 0]
    #print(max_key, test_scores.pop(max_key), len(test_scores))

for k in range(len(top_20)):
    e = top_20[k]
    print("{1:2}. column index: {0[1]:3}     train accuracy: {0[3]:.2f}     test accuracy: {0[4]:5.2f}     column name: {0[2]}".format(e, k+1))
#    print(top_20[k])

In [ ]:
colname = paavo_fs_merged_df.columns[356]
print("Scatterplot between {} and {}".format(colname, "Cafe Summary"))

paavo_fs_merged_df.plot(kind='scatter', x=colname, y='Cafe Summary', figsize=(10,6), color="darkblue")
plt.show()

In [ ]:
colname = paavo_fs_merged_df.columns[49]
print("Scatterplot between {} and {}".format(colname, "Cafe Summary"))

paavo_fs_merged_df.plot(kind='scatter', x=colname, y='Cafe Summary', figsize=(10,6), color="darkblue")
plt.show()